In [1]:
import tqdm
import json
import random
import itertools
import pandas as pd
from typing import List
from collections import Counter, OrderedDict

# Загрузка данных

In [2]:
# Downloading data from github
!git clone https://github.com/dialogue-evaluation/SemSketches.git
%cd SemSketches

Cloning into 'SemSketches'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 83 (delta 24), reused 47 (delta 9), pack-reused 0
Unpacking objects: 100% (83/83), done.
/content/SemSketches


In [ ]:
!ls data

dev  task_2  trial


Для начала, обратимся к демонстрационному набору данных

In [ ]:
CONTEXTS_TRIAL = "data/trial/contexts_trial.data"
SKETCHES_TRIAL = "data/trial/sketches_trial.data"
TRIAL = "data/trial/trial.gold"

Пример скетча

In [ ]:
sketches_dfs = dict()

sketches_trial_data = json.load(open(SKETCHES_TRIAL))
for sketch_name, sketch_columns_di in sketches_trial_data.items():
    colnames = [None] * len(sketch_columns_di)
    columns = [None] * len(sketch_columns_di)
    for colname, (col_ix, col_cells) in sketch_columns_di.items():
        colnames[col_ix] = colname
        columns[col_ix] = col_cells
    columns_ordered_dict = OrderedDict(zip(colnames, columns))
    sketches_dfs[sketch_name] = pd.DataFrame(columns_ordered_dict)

print('\n'.join(str(sketches_dfs).splitlines()[:12]))

{'trial.sketch.rus.0':     Sphere_Special                 Time  ...     ContrAgent Ch_Evaluation
0          в карты            в детстве  ...       с детьми       классно
1        в шахматы  на большой перемене  ...  с мальчишками       неплохо
2         в футбол           по вечерам  ...    с читателем       недурно
3  в азартные игры          каждый день  ...      с собакой   превосходно
4           в игры             допоздна  ...        с сыном      прилично
5         в прятки          в молодости  ...     с ребятами  замечательно

[6 rows x 6 columns], 'trial.sketch.rus.1':                      Locative_FinalPoint  ... Ch_EvaluationOfHumanTemperAndActivity
0                                к людям  ...                           скептически
1                           к литературе  ...                            критически
2                               к стихам  ...                            философски


Предикаты с контекстами

In [ ]:
# Table with context "name", predicate (to be compared with sketch), 
# starting index of the predicate in the sentence,  starting index and sentence itself.
contexts_trial_df = pd.read_json(CONTEXTS_TRIAL).transpose()
contexts_trial_df.head()

,instance,start,end,sentence
trial.sent.rus.0,стать,63,68,"Старый я, песенка моя спета, и нет у меня боль..."
trial.sent.rus.1,вызвала,6,13,Но не вызвала
trial.sent.rus.2,пойдет,40,46,"А может, и вовсе поменяет конституцию и пойдет..."
trial.sent.rus.3,стану,5,10,"Я не стану отрекаться, – объяснил он"
trial.sent.rus.4,становятся,149,159,В этом случае профиль сечения ППЭ реакции зави...


Как должно выглядеть решение задачи:

In [ ]:
# Dictionary with pairs: sentence - sketch
sent2sketch = json.load(open(TRIAL))
print(sent2sketch)

{'trial.sent.rus.0': 'trial.sketch.rus.8', 'trial.sent.rus.1': 'trial.sketch.rus.4', 'trial.sent.rus.2': 'trial.sketch.rus.6', 'trial.sent.rus.3': 'trial.sketch.rus.2', 'trial.sent.rus.4': 'trial.sketch.rus.15', 'trial.sent.rus.5': 'trial.sketch.rus.8', 'trial.sent.rus.6': 'trial.sketch.rus.14', 'trial.sent.rus.7': 'trial.sketch.rus.13', 'trial.sent.rus.8': 'trial.sketch.rus.6', 'trial.sent.rus.9': 'trial.sketch.rus.16', 'trial.sent.rus.10': 'trial.sketch.rus.1', 'trial.sent.rus.11': 'trial.sketch.rus.6', 'trial.sent.rus.12': 'trial.sketch.rus.0', 'trial.sent.rus.13': 'trial.sketch.rus.17', 'trial.sent.rus.14': 'trial.sketch.rus.4', 'trial.sent.rus.15': 'trial.sketch.rus.4', 'trial.sent.rus.16': 'trial.sketch.rus.0', 'trial.sent.rus.17': 'trial.sketch.rus.14', 'trial.sent.rus.18': 'trial.sketch.rus.19', 'trial.sent.rus.19': 'trial.sketch.rus.19', 'trial.sent.rus.20': 'trial.sketch.rus.19', 'trial.sent.rus.21': 'trial.sketch.rus.15', 'trial.sent.rus.22': 'trial.sketch.rus.0', 'trial.sen

# Реализация алгоритма

In [ ]:
sketches_dfs['trial.sketch.rus.0']

,Sphere_Special,Time,Agent,Locative,ContrAgent,Ch_Evaluation
0,в карты,в детстве,дети,на бирже,с детьми,классно
1,в шахматы,на большой перемене,мальчишки,во дворе,с мальчишками,неплохо
2,в футбол,по вечерам,пацаны,на бильярде,с читателем,недурно
3,в азартные игры,каждый день,игроки внизу,на компьютере,с собакой,превосходно
4,в игры,допоздна,ребята,на площадке,с сыном,прилично
5,в прятки,в молодости,команда,на чужом поле,с ребятами,замечательно


Получим список всех возможных ролей с примерами:

In [ ]:
roles = {}
for sketch in sketches_dfs:
    for role in list(sketches_dfs[sketch].columns):
        if role not in roles:
            roles[role] = sketches_dfs[sketch][role][0]
roles

{'Agent': 'дети',
 'Agent_Metaphoric': 'жизнь',
 'Agent_Route': 'дорога',
 'Cause_Actant': 'все пути',
 'Ch_Emotion': 'сухо',
 'Ch_Evaluation': 'классно',
 'Ch_EvaluationOfHumanTemperAndActivity': 'скептически',
 'Ch_PhysicalStateOfObject': 'аккуратно',
 'Ch_PossibilitivityInPerceptionAndRealization': 'понятно',
 'Concurrent_Situative': 'не задумываясь',
 'ContrAgent': 'с детьми',
 'Function': 'свидетелем',
 'Instrument': 'на гитаре',
 'Locative': 'на бирже',
 'LocativeEvent_FinalPoint': 'на дуэль',
 'Locative_Distance': 'ближе',
 'Locative_FinalPoint': 'к людям',
 'Locative_InitialPoint': 'из-за стола',
 'Locative_Orientation_FinalPoint': 'прочь',
 'Locative_Route': 'вдоль стены',
 'MannerOfPositionAndMotion': 'на груди',
 'Metaphoric_FinalPoint': 'в заглавие',
 'Modality': 'вряд ли',
 'Object': 'автор',
 'Object_Situation': 'читать',
 'Object_SituationLike': 'слухи',
 'OrderInTimeAndSpace': 'потом',
 'PartComplement_SituationLike': 'с уважением',
 'Possessor': 'автор',
 'Possessor_Me

Сгенерируем маски с контекстами для предсказаний

In [ ]:
def generate_masked_contexts(sketch: str) -> List[List[str]]:
    masked_contexts = []
    for role in list(sketches_dfs[sketch].columns):
        for word in list(sketches_dfs[sketch][role]):
            if word != '':
                masked_contexts.append('[MASK] ' + word)

    return masked_contexts

In [ ]:
contexts = []
for sketch in sketches_dfs:
    contexts.append(generate_masked_contexts(sketch))

In [ ]:
# пример максированных контекстов
contexts[0]

['[MASK] в карты',
 '[MASK] в шахматы',
 '[MASK] в футбол',
 '[MASK] в азартные игры',
 '[MASK] в игры',
 '[MASK] в прятки',
 '[MASK] в детстве',
 '[MASK] на большой перемене',
 '[MASK] по вечерам',
 '[MASK] каждый день',
 '[MASK] допоздна',
 '[MASK] в молодости',
 '[MASK] дети',
 '[MASK] мальчишки',
 '[MASK] пацаны',
 '[MASK] игроки внизу',
 '[MASK] ребята',
 '[MASK] команда',
 '[MASK] на бирже',
 '[MASK] во дворе',
 '[MASK] на бильярде',
 '[MASK] на компьютере',
 '[MASK] на площадке',
 '[MASK] на чужом поле',
 '[MASK] с детьми',
 '[MASK] с мальчишками',
 '[MASK] с читателем',
 '[MASK] с собакой',
 '[MASK] с сыном',
 '[MASK] с ребятами',
 '[MASK] классно',
 '[MASK] неплохо',
 '[MASK] недурно',
 '[MASK] превосходно',
 '[MASK] прилично',
 '[MASK] замечательно']

Подгружаем Conversational RuBERT для предсказаний

In [20]:
# set up torch `DEVICE` constant
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
# Russian Conversational Bert installation 
!wget -q --show-progress -c "http://files.deeppavlov.ai/deeppavlov_data/bert/ru_conversational_cased_L-12_H-768_A-12_pt.tar.gz"
!tar -zxf ru_conversational_cased_L-12_H-768_A-12_pt.tar.gz

BERT_MODEL_PATH = "ru_conversational_cased_L-12_H-768_A-12_pt"
!cp {BERT_MODEL_PATH}/bert_config.json {BERT_MODEL_PATH}/config.json

ru_conversational_c 100%[===================>] 629.49M  4.14MB/s    in 2m 5s   


In [19]:
!pip install transformers

     |████████████████████████████████| 2.3MB 29.3MB/s 
     |████████████████████████████████| 901kB 37.7MB/s 
     |████████████████████████████████| 3.3MB 31.6MB/s 


In [22]:
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH)
model = BertForMaskedLM.from_pretrained(BERT_MODEL_PATH).to(DEVICE)

Some weights of the model checkpoint at ru_conversational_cased_L-12_H-768_A-12_pt were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Нормализовываем предикаты в предложениях с помощью pymorphy2

In [ ]:
contexts_trial_df.head()

,instance,start,end,sentence
trial.sent.rus.0,стать,63,68,"Старый я, песенка моя спета, и нет у меня боль..."
trial.sent.rus.1,вызвала,6,13,Но не вызвала
trial.sent.rus.2,пойдет,40,46,"А может, и вовсе поменяет конституцию и пойдет..."
trial.sent.rus.3,стану,5,10,"Я не стану отрекаться, – объяснил он"
trial.sent.rus.4,становятся,149,159,В этом случае профиль сечения ППЭ реакции зави...


In [7]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 8.2MB 23.8MB/s 


In [8]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

def tokenize_word(word: str, morph=morph) -> str:
  normalized = morph.parse(word)[0].normal_form
  return normalized

In [ ]:
contexts_trial_df['normalised_instance'] = contexts_trial_df.instance.apply(tokenize_word)
contexts_trial_df.head()

,instance,start,end,sentence,normalised_instance
trial.sent.rus.0,стать,63,68,"Старый я, песенка моя спета, и нет у меня боль...",стать
trial.sent.rus.1,вызвала,6,13,Но не вызвала,вызвать
trial.sent.rus.2,пойдет,40,46,"А может, и вовсе поменяет конституцию и пойдет...",пойти
trial.sent.rus.3,стану,5,10,"Я не стану отрекаться, – объяснил он",стать
trial.sent.rus.4,становятся,149,159,В этом случае профиль сечения ППЭ реакции зави...,становиться


Получаем список кандидатов для каждого скетча и сопоставляем с предикатами в предложениях

In [ ]:
def predict_verb_for_context(context: str) -> List[str]:
  context_ids = tokenizer(context, padding=True, return_tensors="pt")["input_ids"]
  context_tensor = torch.LongTensor(context_ids).to(DEVICE)
  outputs = model(context_tensor)
  mask_token_indices = torch.where(context_tensor == tokenizer.mask_token_id)
  token_logits = outputs['logits']
  indices = torch.topk(token_logits[mask_token_indices],20).indices
  decoded = list(map(tokenizer.convert_ids_to_tokens, indices))
  candidates = []
  for predicted_words in decoded:
    for cand in predicted_words:
      parse_word = morph.parse(cand)[0]
      if parse_word.tag.POS=='VERB':
        candidates.append(parse_word.normal_form)
  return candidates

# Получаем предсказания для тестовых данных

In [3]:
CONTEXTS_DEV = "data/dev/contexts_dev.data"
SKETCHES_DEV = "data/dev/sketches_dev.data"

Загружаются скетчи

In [4]:
sketches_dev_dfs = dict()

sketches_dev_data = json.load(open(SKETCHES_DEV))
for sketch_name, sketch_columns_di in sketches_dev_data.items():
    colnames = [None] * len(sketch_columns_di)
    columns = [None] * len(sketch_columns_di)
    for colname, (col_ix, col_cells) in sketch_columns_di.items():
        colnames[col_ix] = colname
        columns[col_ix] = col_cells
    columns_ordered_dict = OrderedDict(zip(colnames, columns))
    sketches_dev_dfs[sketch_name] = pd.DataFrame(columns_ordered_dict)

print('\n'.join(str(sketches_dev_dfs).splitlines()[:12]))

{'dev.sketch.rus.0':          Object_Situation  ...  DegreeNumerative
0             число газет  ...   примерно на 60%
1    количество библиотек  ...          на треть
2     численность занятых  ...  на 53,7 процента
3                   объем  ...      один миллион
4        посевная площадь  ...             людей
5  доля голосов епископов  ...                  
6   расстояние между нами  ...                  
7                    риск  ...                  

[8 rows x 6 columns], 'dev.sketch.rus.1':          Locative_Route  ... Locative_Orientation_FinalPoint
0  вдоль большой дороги  ...                       навстречу


Загружаются предложения

In [5]:
contexts_dev_df = pd.read_json(CONTEXTS_DEV).transpose()
contexts_dev_df.head()

,instance,start,end,sentence
dev.sent.rus.0,отбирать,169,177,"Люди, производящие очень дорогостоящее оборудо..."
dev.sent.rus.1,пожал,44,49,Он не спеша подошел к полковнику Эмсуорту и по...
dev.sent.rus.2,обратил,289,296,Докладчик также представил отчет о рассмотрени...
dev.sent.rus.3,остановился,32,43,"Но, пройдя несколько шагов, остановился и..."
dev.sent.rus.4,выделил,80,87,"Чтобы предотвратить это, Центральный фонд реаг..."


In [ ]:
contexts_dev_df

In [ ]:
sketches_dev_dfs['dev.sketch.rus.0']

,Object_Situation,DegreeIntensity,Time,DegreeMultiplicative,Object,DegreeNumerative
0,число газет,значительно,в посткризисный период,вдвое,государственные расходы,примерно на 60%
1,количество библиотек,резко,постоянно,в разы,факторные издержки производства,на треть
2,численность занятых,существенно,одновременно,,население,"на 53,7 процента"
3,объем,ничуть,к концу века,,налогооблагаемый доход,один миллион
4,посевная площадь,сильно,с годами,,сумма нашего долга,людей
5,доля голосов епископов,чуть,за последние полвека,,тиражи поэтических книг,
6,расстояние между нами,немного,до 500 г,,срок,
7,риск,и более,на какое-то время,,налоги,


Выведем роли, соответствующие dev данным

In [23]:
roles = {}
for sketch in sketches_dev_dfs:
    for role in list(sketches_dev_dfs[sketch].columns):
        if role not in roles:
            roles[role] = sketches_dev_dfs[sketch][role][0]
roles

{'Addition': 'также',
 'AdditiveDegree': 'еще',
 'AdditiveQuantity': 'еще',
 'Addressee': 'врагу',
 'Addressee_Metaphoric': 'своим глазам',
 'Agent': 'коровы',
 'Agent_Device': 'поезд',
 'Agent_Metaphoric': 'годы',
 'Agent_Route': 'улица',
 'AttachedProperty': 'в союзники',
 'BehalfOfEntity': 'от первого лица',
 'BeneMalefactive': 'русскому читателю',
 'Causator': 'мать',
 'Cause': 'по какой-то причине',
 'Cause_Actant': 'жизнью',
 'Cause_From': 'от страха',
 'ChRelation_Grade': 'на три',
 'Ch_Ability': 'проницательно',
 'Ch_AvailabilityForUse': 'свободно',
 'Ch_ByLanguage': 'на русском языке',
 'Ch_Emotion': 'нежно',
 'Ch_Evaluation': 'удачно',
 'Ch_EvaluationOfHumanTemperAndActivity': 'от души',
 'Ch_Exactness': 'неточно',
 'Ch_Information': 'подробно',
 'Ch_NotabilityDistribution': 'торжественно',
 'Ch_Parameter_Density': 'неплотно',
 'Ch_Parameter_Price': 'дорого',
 'Ch_Parameter_Speed': 'быстро',
 'Ch_Perceptibility': 'резко',
 'Ch_PerceptionAndViews': 'радикально',
 'Ch_Phys_Form

In [9]:
contexts_dev_df['normalised_instance'] = contexts_dev_df.instance.apply(tokenize_word)
contexts_dev_df.head()

,instance,start,end,sentence,normalised_instance
dev.sent.rus.0,отбирать,169,177,"Люди, производящие очень дорогостоящее оборудо...",отбирать
dev.sent.rus.1,пожал,44,49,Он не спеша подошел к полковнику Эмсуорту и по...,пожать
dev.sent.rus.2,обратил,289,296,Докладчик также представил отчет о рассмотрени...,обратить
dev.sent.rus.3,остановился,32,43,"Но, пройдя несколько шагов, остановился и...",остановиться
dev.sent.rus.4,выделил,80,87,"Чтобы предотвратить это, Центральный фонд реаг...",выделить


Сгенерируем шаблоны

In [24]:
# пытаемся сделать контексты вида Agent + MASK + другая роль
def generate_masked_contexts_dev(sketch: str) -> List[List[str]]:
    agents = []
    other_roles = []
    for role in list(sketches_dev_dfs[sketch].columns):
        for word in list(sketches_dev_dfs[sketch][role]):
            if word != '':
                if role.split('_')[0] == 'Agent':
                    agents.append(word)
                else:
                    other_roles.append(word)
    if agents == []:
        agents = ['он', 'они']
    role_pairs = list(itertools.product(agents, other_roles))
    masked_contexts = [pair[0] + ' [MASK] ' + pair[1] for pair in role_pairs]
    return masked_contexts

In [25]:
# делаем попарные контексты
def generate_masked_contexts_dev_pairs(sketch: str) -> List[List[str]]:
    masked_contexts = []
    for role in list(sketches_dev_dfs[sketch].columns):
        for word in list(sketches_dev_dfs[sketch][role]):
            if word != '' and role.split('_')[0] != 'Agent':
            #if word != '':
                masked_contexts.append('[MASK] ' + word)
                masked_contexts.append(word + ' [MASK]')
    return masked_contexts

In [26]:
contexts_dev = []
for sketch in sketches_dev_dfs:
    contexts_dev.append(generate_masked_contexts_dev(sketch) + generate_masked_contexts_dev_pairs(sketch))

In [ ]:
contexts_dev[0]

['он [MASK] число газет',
 'он [MASK] количество библиотек',
 'он [MASK] численность занятых',
 'он [MASK] объем',
 'он [MASK] посевная площадь',
 'он [MASK] доля голосов епископов',
 'он [MASK] расстояние между нами',
 'он [MASK] риск',
 'он [MASK] значительно',
 'он [MASK] резко',
 'он [MASK] существенно',
 'он [MASK] ничуть',
 'он [MASK] сильно',
 'он [MASK] чуть',
 'он [MASK] немного',
 'он [MASK] и более',
 'он [MASK] в посткризисный период',
 'он [MASK] постоянно',
 'он [MASK] одновременно',
 'он [MASK] к концу века',
 'он [MASK] с годами',
 'он [MASK] за последние полвека',
 'он [MASK] до 500 г',
 'он [MASK] на какое-то время',
 'он [MASK] вдвое',
 'он [MASK] в разы',
 'он [MASK] государственные расходы',
 'он [MASK] факторные издержки производства',
 'он [MASK] население',
 'он [MASK] налогооблагаемый доход',
 'он [MASK] сумма нашего долга',
 'он [MASK] тиражи поэтических книг',
 'он [MASK] срок',
 'он [MASK] налоги',
 'он [MASK] примерно на 60%',
 'он [MASK] на треть',
 'он [M

Основная функция для заполнения маски BERTом

In [27]:
def predict_verb_for_context(context: str) -> List[str]:
  context_ids = tokenizer(context, padding=True, return_tensors="pt")["input_ids"]
  context_tensor = torch.LongTensor(context_ids).to(DEVICE)
  outputs = model(context_tensor)
  mask_token_indices = torch.where(context_tensor == tokenizer.mask_token_id)
  token_logits = outputs['logits']
  indices = torch.topk(token_logits[mask_token_indices],10).indices
  decoded = list(map(tokenizer.convert_ids_to_tokens, indices))
  candidates = []
  for predicted_words in decoded:
    for cand in predicted_words:
      parse_word = morph.parse(cand)[0]
      if parse_word.tag.POS == 'VERB':
        candidates.append(parse_word.normal_form)
  return candidates

Пример предсказания для скетча

In [ ]:
contexts_dev[5]

['толпа [MASK] на площади',
 'толпа [MASK] в зале',
 'толпа [MASK] за столом',
 'толпа [MASK] в клубе',
 'толпа [MASK] в квартире',
 'толпа [MASK] у кого-нибудь дома',
 'толпа [MASK] под одной обложкой',
 'толпа [MASK] в комнате',
 'толпа [MASK] вечером',
 'толпа [MASK] по воскресеньям',
 'толпа [MASK] по субботам',
 'толпа [MASK] в 9 часов',
 'толпа [MASK] регулярно',
 'толпа [MASK] по выходным',
 'толпа [MASK] утром',
 'толпа [MASK] по пятницам',
 'толпа [MASK] в дорогу',
 'толпа [MASK] в кучу',
 'толпа [MASK] в стаи',
 'толпа [MASK] в кучку',
 'толпа [MASK] в кружок',
 'толпа [MASK] под одно знамя',
 'толпа [MASK] в единое целое',
 'толпа [MASK] в школу',
 'толпа [MASK] вновь',
 'толпа [MASK] снова',
 'толпа [MASK] опять',
 'толпа [MASK] в последний раз',
 'толпа [MASK] наконец',
 'толпа [MASK] сперва',
 'толпа [MASK] заново',
 'толпа [MASK] впервые',
 'толпа [MASK] воедино',
 'толпа [MASK] вместе',
 'компания [MASK] на площади',
 'компания [MASK] в зале',
 'компания [MASK] за столо

Функция ранжирования предсказаний по частотности

In [28]:
def get_verb_prediction(cont):
    verb_counter = dict(Counter(predict_verb_for_context(cont)))
    for verb in verb_counter:
        verb_counter[verb] /= len(cont)
    return verb_counter

In [32]:
get_verb_prediction(contexts_dev[5])

{'##быть': 0.0058823529411764705,
 '##иться': 0.011764705882352941,
 '##никнуть': 0.0029411764705882353,
 '##нить': 0.020588235294117647,
 '##нуться': 0.0029411764705882353,
 '##ть': 0.011764705882352941,
 'быть': 0.42058823529411765,
 'ввести': 0.0058823529411764705,
 'вернуться': 0.014705882352941176,
 'взорваться': 0.0029411764705882353,
 'включить': 0.014705882352941176,
 'войти': 0.06764705882352941,
 'встать': 0.020588235294117647,
 'встереть': 0.0058823529411764705,
 'встретиться': 0.0058823529411764705,
 'вступить': 0.0029411764705882353,
 'входить': 0.061764705882352944,
 'въехать': 0.0029411764705882353,
 'выехать': 0.008823529411764706,
 'выйти': 0.008823529411764706,
 'гулять': 0.014705882352941176,
 'дать': 0.008823529411764706,
 'действовать': 0.023529411764705882,
 'добавить': 0.0029411764705882353,
 'дойти': 0.0058823529411764705,
 'ездить': 0.03529411764705882,
 'есть': 0.058823529411764705,
 'ехать': 0.06764705882352941,
 'ждать': 0.1323529411764706,
 'жечь': 0.008823

Для каждого набора скетчей получаем предсказания:

In [ ]:
from tqdm.notebook import tqdm

sketches_pred = {}
for i in tqdm(range(len(contexts_dev))):
    sketches_pred['dev.sketch.rus.' + str(i)] = get_verb_prediction(contexts_dev[i])

Фиксируем в файл

In [ ]:
with open("data_file_2.json", "w") as write_file:
    json.dump(sketches_pred, write_file, ensure_ascii=False)

#Подготовка данных для загрузки

Загружаем пакет gensim и модель word2vec для поиска синонимов

In [ ]:
!pip install gensim --upgrade -q

     |████████████████████████████████| 24.2MB 9.3MB/s 


In [ ]:
import gensim

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/180.zip
!unzip 180.zip

--2021-03-21 01:38:17--  http://vectors.nlpl.eu/repository/20/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484452317 (462M) [application/zip]
Saving to: ‘180.zip’

180.zip             100%[===================>] 462.01M  97.6MB/s    in 4.9s    

2021-03-21 01:38:22 (93.9 MB/s) - ‘180.zip’ saved [484452317/484452317]

Archive:  180.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

Функция для поиска синонимов

In [ ]:
def find_synonims(verb, model=model):
  verb = verb+'_VERB'
  top_syms = model.most_similar(verb, topn=10)
  top_syms_verb = []
  for sym,_ in top_syms:
    if sym.split('_')[1]=='VERB':
      top_syms_verb.append(sym.split('_')[0])
  return top_syms_verb

Пример работы:

In [ ]:
find_synonims('кричать')

['орать',
 'закричать',
 'вопить',
 'крикнуть',
 'выкрикивать',
 'реветь',
 'визжать',
 'прокричать',
 'заорать']

Функция для соотнесения предложений со скетчами

In [ ]:
def sketch_by_verb(sketches_pred, verb):
    share = 0
    num_sketch = 'dev.sketch.rus.' + str(random.randint(0, 894))
    for sketch in sketches_pred:
        if verb in sketches_pred[sketch]:
            if sketches_pred[sketch][verb] > share:
                share = sketches_pred[sketch][verb]
                num_sketch = sketch
    return num_sketch, share

Основной цикл, где происходит сопоставление предложений со скетчами, внутри которого поиск по синонимам

In [ ]:
data = {}
for sent in tqdm(list(contexts_dev_df.index)):
    num_sketch, share = sketch_by_verb(res, contexts_dev_df['normalised_instance'][sent])
    if not share:
        syns = find_synonims
        syn_ind = 0
        while not share:
            num_sketch, share = sketch_by_verb(res, syns[syn_ind])
            syn_ind += 1
            if syn_ind == len(syn):
                break
    data[sent] = num_sketch

In [ ]:
with open("task1.json", "w") as write_file:
    json.dump(data, write_file, ensure_ascii=False)